In [1]:
import pandas as pd
import tensorflow as tf
import os

try:
    from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

except:
    !pip install transformers
    from transformers import AutoTokenizer, TFAutoModelForSequenceClassification


# Clone GH for access to data
!git clone https://github.com/pvankatwyk/fake-reviews-classification

# Move model.py file to root directory so you can call using "from model import RobertaModel"
try:
    os.rename("/content/fake-reviews-classification/our code/model.py", "/content/model.py")
except FileNotFoundError:
    pass

     |████████████████████████████████| 4.0 MB 5.6 MB/s 
     |████████████████████████████████| 77 kB 6.9 MB/s 
     |████████████████████████████████| 6.6 MB 40.6 MB/s 
     |████████████████████████████████| 596 kB 55.8 MB/s 
     |████████████████████████████████| 880 kB 5.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=b4049d3e0a13203e34e66a1fbab85e6a5934abefd266e7db0478c2de641a2349
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'fake-reviews-classification'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 32 (delta 10), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% 

In [2]:
encoded_label_dict = {"CG" : 0, "OR" : 1}
def encode_label(x):
    return encoded_label_dict.get(x,-1)

df = pd.read_csv("/content/fake-reviews-classification/data/fake_reviews_dataset.csv")
df["target"] = df["label"].apply(lambda x: encode_label(x))

In [3]:
model_name = "roberta-base"
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 1
LEARNING_RATE = 1e-05

In [4]:
texts = list(df['text_'])
labels = list(df['target'])

In [5]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base") #Tokenizer
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='tf') #Tokenized text

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [6]:
dataset=tf.data.Dataset.from_tensor_slices((dict(inputs), labels)) #Create a tensorflow dataset
#train test split, we use 10% of the data for validation
val_data_size=int(0.1*len(dataset))
val_ds=dataset.take(val_data_size).batch(TRAIN_BATCH_SIZE, drop_remainder=True) 
train_ds=dataset.skip(val_data_size).batch(TRAIN_BATCH_SIZE, drop_remainder=True)

In [7]:
n_categories = 2

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=n_categories)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, clipnorm=1.),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name='Sparse_Top_3_Categorical_Accuracy')],
    )
 
h=model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, verbose=1)

model.save_weights('./saved_weights.h5')

Downloading:   0%|          | 0.00/627M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1847/4548 [===========>..................] - ETA: 42:55 - loss: 0.2044 - sparse_categorical_accuracy: 0.9138 - Sparse_Top_3_Categorical_Accuracy: 1.0000

In [ ]:
h